In [1]:
import numpy as np

from stochastic_volatility_models.src.core.underlying import Underlying
from stochastic_volatility_models.src.core.volatility_surface import VolatilitySurface
from stochastic_volatility_models.src.core.pricing_models import PricingModel
from stochastic_volatility_models.src.models.heston import HestonModel

2024-07-31 03:48:54.728 | INFO     | stochastic_volatility_models.config:initialise:30 - Loaded module `stochastic_volatility_models` from project directory path `/Users/mayurankv/Documents/Mayuran/Programming/Projects/Academic/Imperial College London/MSc Statistics/Dissertation/Project/modules`


In [2]:
ticker = "SPX"
spx = Underlying(ticker)
vix = Underlying("VIX")
pricing_model = PricingModel()
time = np.datetime64("2022-03-03")

In [3]:
expiries = np.array(
	["2022-03-04", "2022-03-09", "2022-03-11", "2022-03-18", "2022-03-23", "2022-03-25", "2022-03-30", "2022-03-31", "2022-04-01", "2022-04-08", "2022-04-14", "2022-04-22", "2022-04-29", "2022-05-20", "2022-05-31", "2022-06-17", "2022-06-30", "2022-07-15", "2022-07-29", "2022-08-31"], dtype=np.datetime64
)

strikes = np.array(
	[
		2200,
		2400,
		2600,
		2800,
		3000,
		3200,
		3400,
		3500,
		3600,
		3700,
		3800,
		3850,
		3900,
		3950,
		3975,
		4000,
		4025,
		4040,
		4050,
		4060,
		4070,
		4075,
		4080,
		4090,
		4100,
		4110,
		4120,
		4125,
		4130,
		4140,
		4150,
		4160,
		4170,
		4175,
		4180,
		4190,
		4200,
		4210,
		4220,
		4225,
		4230,
		4240,
		4250,
		4260,
		4270,
		4275,
		4280,
		4290,
		4300,
		4310,
		4320,
		4325,
		4330,
		4340,
		4350,
		4360,
		4370,
		4375,
		4380,
		4390,
		4400,
		4410,
		4420,
		4425,
		4430,
		4440,
		4450,
		4460,
		4470,
		4475,
		4480,
		4490,
		4500,
		4510,
		4525,
		4550,
		4600,
		4650,
		4700,
		4800,
		5000,
		5200,
		5400,
	]
)
spx_vs = VolatilitySurface(
	underlying=spx,
	expiries=expiries,
	strikes=strikes,
	monthly=False,
)

In [4]:
expiries = np.array(["2022-03-09", "2022-03-23", "2022-03-30", "2022-04-06"], dtype=np.datetime64)

strikes = np.array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32.5, 35, 37.5, 40, 42.5, 45, 47.5, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100])
vix_vs = VolatilitySurface(
	underlying=vix,
	expiries=expiries,
	strikes=strikes,
	monthly=False,
)

In [5]:
model = HestonModel(
	parameters={
		"initial_variance": 0.05,
		"long_term_variance": 0.05,
		"volatility_of_volatility": 0.3,
		"mean_reversion_rate": 2.0,
		"wiener_correlation": -0.5,
	}
)

In [6]:
from typing import Callable
import numpy as np
from numpy.typing import NDArray
from numba import jit

DEFAULT_LG_DEGREE = 40


@jit
def characteristic_function(
	u: complex | NDArray[np.complex64],
	spot: float,
	time_to_expiry: float,
	risk_free_rate: float,
	dividend_yield: float,
	initial_variance: float,
	long_term_variance: float,
	volatility_of_volatility: float,
	mean_reversion_rate: float,
	wiener_correlation: float,
) -> complex | NDArray[np.complex64]:
	F = spot * np.exp((risk_free_rate - dividend_yield) * time_to_expiry)
	xi = mean_reversion_rate - volatility_of_volatility * wiener_correlation * 1j * u
	d = np.sqrt(xi**2 + (u**2 + 1j * u) * volatility_of_volatility**2)
	A1 = (u**2 + u * 1j) * np.sinh(d * time_to_expiry / 2)
	A2 = (d * np.cosh(d * time_to_expiry / 2) + xi * np.sinh(d * time_to_expiry / 2)) / initial_variance
	A = A1 / A2
	D = np.log(d / initial_variance) + (mean_reversion_rate - d) * time_to_expiry / 2 - np.log(((d + xi) + (d - xi) * np.exp(-d * time_to_expiry)) / (2 * initial_variance))
	value = np.exp(1j * u * np.log(F / spot) - mean_reversion_rate * long_term_variance * wiener_correlation * time_to_expiry * 1j * u / volatility_of_volatility - A + 2 * mean_reversion_rate * long_term_variance * D / (volatility_of_volatility**2))

	return value


def lg_integrate(
	integrand: Callable[[NDArray[np.float64]], NDArray[np.float64]],
	degree: int = DEFAULT_LG_DEGREE,
) -> float:
	def transformed_integrand(
		u: NDArray[np.float64],
	) -> NDArray[np.float64]:
		t = u / (1 - u)
		return integrand(t) / (1 - u) ** 2

	nodes, weights = np.polynomial.legendre.leggauss(deg=degree)
	nodes = 0.5 * (nodes + 1)
	weights = 0.5 * weights
	approximation = sum(weights * transformed_integrand(u=nodes))

	return approximation


@jit
def p1_value(
	spot: float,
	strike: int,
	time_to_expiry: float,
	risk_free_rate: float,
	dividend_yield: float,
	initial_variance: float,
	long_term_variance: float,
	volatility_of_volatility: float,
	mean_reversion_rate: float,
	wiener_correlation: float,
	degree: int = DEFAULT_LG_DEGREE,
) -> float:
	def integrand(
		u: NDArray[np.float64],
	) -> NDArray[np.float64]:
		return np.real(
			(np.exp(-1j * u * np.log(strike / spot)) / (u * 1j))
			* (
				characteristic_function(
					u=u - 1j,
					spot=spot,
					time_to_expiry=time_to_expiry,
					risk_free_rate=risk_free_rate,
					dividend_yield=dividend_yield,
					initial_variance=initial_variance,
					long_term_variance=long_term_variance,
					volatility_of_volatility=volatility_of_volatility,
					mean_reversion_rate=mean_reversion_rate,
					wiener_correlation=wiener_correlation,
				)
				/ characteristic_function(
					u=-1j,
					spot=spot,
					time_to_expiry=time_to_expiry,
					risk_free_rate=risk_free_rate,
					dividend_yield=dividend_yield,
					initial_variance=initial_variance,
					long_term_variance=long_term_variance,
					volatility_of_volatility=volatility_of_volatility,
					mean_reversion_rate=mean_reversion_rate,
					wiener_correlation=wiener_correlation,
				)
			)
		)

	integral = lg_integrate(
		integrand=integrand,
		degree=degree,
	)

	value = integral / np.pi + 1 / 2

	return value

In [7]:
p1_value(
	spx.price(time),
	spx_vs.strikes[0],
	0.1,
	0.01,
	0.01,
	**model.parameters,
)

TypingError: Failed in nopython mode pipeline (step: convert make_function into JIT functions)
Cannot capture the non-constant value associated with variable 'dividend_yield' in a function that may escape.

File "../../../../../../../../../../../../var/folders/5x/hwwm8kmn4z77258t79_46vfm0000gn/T/ipykernel_28860/3286786197.py", line 66:
<source missing, REPL/exec in use?>


In [6]:
spx_vs.model_price(time, model)

TypingError: Failed in nopython mode pipeline (step: convert make_function into JIT functions)
Cannot capture the non-constant value associated with variable 'dividend_yield' in a function that may escape.

File "../modules/stochastic_volatility_models/stochastic_volatility_models/src/models/heston.py", line 128:
) -> float:
	def integrand(
 ^


In [ ]:
model.fit(
	index_volatility_surface=spx_vs,
	volatility_index_volatility_surface=vix_vs,
	time=time,
	pricing_model=PricingModel(),
)

/Users/mayurankv/Documents/Mayuran/Programming/Projects/Academic/Imperial College London/MSc Statistics/Dissertation/.venv/lib/python3.12/site-packages/py_vollib_vectorized/implied_volatility.py:75: UserWarning: Found Below Intrinsic contracts at index [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102, 104, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138, 140, 160, 162, 164, 166, 168, 170, 172, 174, 176, 178, 180, 200, 202, 204, 206, 208, 240, 242, 244, 246, 280, 282, 284, 286, 320, 322, 324, 360, 362, 400, 402, 440, 480, 520, 560, 600, 640, 680, 720, 760, 800, 840, 880, 920, 960, 1000, 1040, 1080, 1120, 1160, 1200, 1240, 1280, 1320, 1360, 1400, 2681, 2721, 2761, 2801, 2841, 2881, 2921, 2961, 3001, 3003, 3005, 3041, 3043, 3081, 3083, 3085, 3087, 3121, 3123, 3125, 3127, 3129, 3131, 3161, 3163, 3165, 3167, 3169, 3171, 3173, 3175, 3177, 3179, 3181, 3201, 3203, 3205, 3207, 3209, 3211, 32

TypingError: Failed in nopython mode pipeline (step: convert make_function into JIT functions)
Cannot capture the non-constant value associated with variable 'option_parameters' in a function that may escape.

File "../modules/stochastic_volatility_models/stochastic_volatility_models/src/models/heston.py", line 87:
	def price(
     <source elided>
		integral, _ = quad(
			func=lambda x: np.real(
   ^ 

This error may have been caused by the following argument(s):
- argument 0: Cannot determine Numba type of <class 'stochastic_volatility_models.src.models.heston.HestonModel'>
- argument 1: Cannot determine Numba type of <class 'stochastic_volatility_models.src.core.underlying.Underlying'>
- argument 3: Cannot determine Numba type of <class 'dict'>
